# Decorators

## Introduction

- Decorators is a function that takes anothor function as input and returns a new function that usually extends or alters the behaviour of original one. We can simply say decorator are a way to modify or enhance the behaviour of functions or classes without changing the code.

- Recall the closure example where we have calculated the no of times the function gets called :

  ```python

  def counter(fn, counters):

    count = 0

    def inner(*args,**kwargs):

        nonlocal count

        count += 1

        counters[fn.__name__] = count

        return fn(*args,**kwargs)
    
    return inner

  def add(a,b):

    return a + b

  add = counter(add)

  result = add(1,2)

  ```

  Here we have modified our add function by wrapping it inside the counter function that added the functionality which is counting the no of times add functions gets called. Since counter function changed the functionality of the add function, then we can say `counter` is called as
  Decorator.

- In general a decorator function : 

  1. takes function as argument
  2. returns a closure
  3. the closure usually accepts any combination of parameters
  4. runs some code in the inner function (closure)
  5. the closure function calls the original function using the arguments passed to the closure
  6. returns whatever is returned by the function call.

- So in above example, we say `counter` is a decorator. In general, if `func` is a decorator function, we decorate another function `my_func` using `my_func = func(my_func)`. But python provides another way to decorate a function (not by the assignment we have seen just now.)

  ```python

  @counter
  def add(a,b):

    return a + b
  
  ```

  The above code is same as this

  ```python

  def add(a,b):

    return a + b

  add = counter(add)

  ```

  So we can use `@decorator_function_name` before the function that we actually wanna decorate instead of using that assignment operation we have seen.

In [1]:
# Now lets see the decorators in practice

def add(a,b):

    return a + b

add(1,2)

3

In [3]:
# Now we actually wanna change the functionality of add function, so that it should result the how many times this function gets called.

# For this iam writing a function called counter

def counter(fn):

    count = 0

    def inner(*args,**kwargs):

        nonlocal count

        count += 1

        print(f'Function {fn.__name__} gets called {count} times')
        return fn(*args,**kwargs)
    
    return inner

In [4]:
# Now we call the counter by passing the add function. So it would return a closure which contain the function inner

add = counter(add)

# Now we can try this closure is working correctly or not which means it is performing the add functionality or not.

add(1,2)

Function add gets called 1 times


3

In [7]:
# If you see the retuned closure performing addition operation well.

# As we know instead of using the assignment operation we can use @ for decorating a function

@counter

def add(a,b = 1):

    """
    This add function takes two arguments a and b. This will return addition of a and b.
    """

    return a + b

add(5,6)

Function add gets called 1 times


11

In [8]:
# If you see the above output, the functionality of add gets changed because of the counter decorator.

# But we have a problem here. SInce we have created the counter and add function we know which parameters we need to pass here. 

# But when some other person using this closure, he might not know which parameters need to pass for add function. SO he will use help function to get the function info.

help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [9]:
# But if you see the output , it doesn't contain the signature of the add function. Because from counter function we actually get a closure 
# which contain the function called inner and this add variable references the closure object not actual add function.

# So when we run the help function, it actually returns the doc string or annotations of inner function not add function. This might be a confusion
# when a new user will see this.

# So to replace the signature and annotations of closure (or inner) with the function we actually passing (here passing function is add), we use 
# another decorator called wraps which this passing function as input and decorates the inner function by replacing the singnature, docstring, 
# annotations of inner with the passing function (here it is add).

from functools import wraps

def counter(fn):

    count = 0

    @wraps(fn) # Here it is equivalent to inner = wraps(fn)(inner). We will see these kind of decorators in parameterized decorators.
    def inner(*args,**kwargs):

        nonlocal count

        count += 1

        print(f'Function {fn.__name__} gets called {count} times')

        return fn(*args,**kwargs)
    
    return inner

In [10]:

@counter

def add(a,b = 1):

    """
    This add function takes two arguments a and b. This will return addition of a and b.
    """

    return a + b

add(5,6)

Function add gets called 1 times


11

In [ ]:
# Now lets see the signature of this add function using help function

help(add)

# Here we can see the wraps decorator overwritten the signature of inner with signature of add.

Help on function add in module __main__:

add(a, b=1)
    This add function takes two arguments a and b. This will return addition of a and b.



## Application 1 : Timer

In [1]:
# 1) Timer

# It actually finds the time taken by each function to gets executed

def timed(fn):

    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args,**kwargs):

        start = perf_counter()

        result = fn(*args,**kwargs)

        end = perf_counter()

        elapsed = end - start

        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k,v) for (k,v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)

        print('{0}({1}) took {2:.6f}s to run'.format(fn.__name__,args_str,elapsed))

        return result
    return inner

In [ ]:
# Now lets test the fibbonacci function runtime written in 3 different ways. One is recursive, loop and using reduce function.

@timed
def fibonacci_rescursive(n):

    if n<=2:
        return 1
    return fibonacci_rescursive(n-1) + fibonacci_rescursive(n-2)

In [3]:
fibonacci_rescursive(4)

fibonacci_rescursive(2) took 0.000001s to run
fibonacci_rescursive(1) took 0.000002s to run
fibonacci_rescursive(3) took 0.000708s to run
fibonacci_rescursive(2) took 0.000001s to run
fibonacci_rescursive(4) took 0.000756s to run


3

In [4]:
# Here we have problem, we are  getting the additional runtimes along with actual runtime for finding the fibonacci number 4.
# This is because decorator function gets applied for all the internal recursive functions.

# But we want only time taken for finding the fibbonacci number 4. So to get this, we use another function which calls this fibbonacci function.

def fibonacci_rescursive(n):

    if n<=2:
        return 1
    return fibonacci_rescursive(n-1) + fibonacci_rescursive(n-2)

@timed

def recusive_fibonacci(n):
    return fibonacci_rescursive(n)

In [5]:
recusive_fibonacci(5)

recusive_fibonacci(5) took 0.000005s to run


5

In [7]:
recusive_fibonacci(35)

recusive_fibonacci(35) took 3.075924s to run


9227465

In [8]:
recusive_fibonacci(36)

recusive_fibonacci(36) took 4.553590s to run


14930352

In [9]:
# Now lets check the runtime of fibonacci using the loop

@timed

def fibonacci_loop(n):

    fib_1 = 1
    fib_2 = 1

    for i in range(3,n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    
    return fib_2

In [10]:
fibonacci_loop(4)

fibonacci_loop(4) took 0.000003s to run


3

In [11]:
fibonacci_loop(35)

fibonacci_loop(35) took 0.000007s to run


9227465

In [ ]:
fibonacci_loop(36)

# If you see fibonacci_loop runs faster than recursive function

fibonacci_loop(36) took 0.000008s to run


14930352

In [ ]:
# Now lets do this by using reduce function

from functools import reduce
@timed
def fibonacci_reduce(n):

    fib = reduce(lambda prev,n : (prev[0]+prev[1],prev[0]),range(0,n),(1,0))
    
    return fib[0]

In [16]:
fibonacci_reduce(4)

fibonacci_reduce(4) took 0.000007s to run


5

In [17]:
fibonacci_reduce(35)

fibonacci_reduce(35) took 0.000019s to run


14930352

In [19]:
fibonacci_reduce(36)

fibonacci_reduce(36) took 0.000013s to run


24157817

In [26]:
# Generally if you run same code mutiple times it would give different time becuase of process in cpu. So instead of absolute difference, here we need to consider average elapsed time of certain number iterations.

# So here iam changing the code of timed decorator to incoperate this.

def timed(fn):

    from time import perf_counter
    from functools import wraps


    @wraps(fn)
    def inner(*args,**kwargs):

        elapsed_total = 0
        elapsed_count = 0

        for i in range(10):

            print(f"Running iteration {i} .....")

            start = perf_counter()

            result = fn(*args,**kwargs)

            end = perf_counter()

            elapsed = end - start

            elapsed_total += elapsed

            elapsed_count += 1

        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k,v) for (k,v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)

        elapsed_avg = elapsed_total/elapsed_count

        print('{0}({1}) took {2:.6f}s to run'.format(fn.__name__,args_str,elapsed_avg))

        return result
    return inner

In [27]:

from functools import reduce
@timed
def fibonacci_reduce(n):

    fib = reduce(lambda prev,n : (prev[0]+prev[1],prev[0]),range(0,n),(1,0))
    
    return fib[0]

In [ ]:
fibonacci_reduce(35)

# Here we are hardcoding the value in for loop. Instead we wanna pass it as value to counter, so that we can find accurate runtime.

# But we cannot use @ notation for creating decorated function with parameters. We need to use parameterized decorators for it.

Running iteration 0 .....
Running iteration 1 .....
Running iteration 2 .....
Running iteration 3 .....
Running iteration 4 .....
Running iteration 5 .....
Running iteration 6 .....
Running iteration 7 .....
Running iteration 8 .....
Running iteration 9 .....
fibonacci_reduce(35) took 0.000009s to run


14930352

## Application 2 : Logging and Stacked Decorators

- Stacking decorators mean we actually applying one decorator after anothor. Consider the following example here :

  ```python

  @dec_1
  @dec_2

  def my_func():
    pass

  ```

  Here we actually applying decorator 1 first and passing the inner function of decorator 2 to decorator 1. The above code is simply equivalent to `my_func = dec_1(dec_2(my_func))`. So we simply passing my_func to dec_2 first and then the resultant closure is again decorated by the decorator 1 which is dec_1.

In [63]:
# Now lets see te application of logging. Logging simply means when the function gets called and that info must be stored in seperate files.
# But here we are just printing it.

def logged(fn):

    from functools import wraps
    from datetime import datetime , timezone, timedelta
    @wraps(fn)
    def inner(*args,**kwargs):

        run_dt = datetime.now(timezone(timedelta(hours = -4)))
        result = fn(*args,**kwargs)
        print('{0} called {1}'.format(run_dt, fn.__name__))
        return result
    return inner

In [64]:
# For explain the stacked decorators iam just usign timer decorator again.

def timed(fn):

    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args,**kwargs):

        start = perf_counter()

        result = fn(*args,**kwargs)

        end = perf_counter()

        elapsed = end - start

        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k,v) for (k,v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)

        print('{0}({1}) took {2:.6f}s to run'.format(fn.__name__,args_str,elapsed))

        return result
    return inner

In [65]:
# Now iam defining the factorial function using reduce

def fact(n):

    from operator import mul
    from functools import reduce

    return reduce(mul,range(1,n+1))
fact(5)

120

In [66]:
# Now iam decorating it using logged function

@logged

def fact(n):

    from operator import mul
    from functools import reduce

    return reduce(mul,range(1,n+1))

fact(5)


2025-06-17 11:56:30.076660-04:00 called fact


120

In [ ]:
# Now lets decorate the fact function using both logged and timed one.

@logged

@timed

def fact(n):

    from operator import mul
    from functools import reduce

    return reduce(mul,range(1,n+1))

fact(5)

# Here it is equivalent to fact = logged(timed(fact))

fact(5) took 0.000011s to run
2025-06-17 11:56:35.377179-04:00 called fact


120

In [ ]:
# See the difference if written decorators in reverse order 

@timed
@logged
def fact(n):

    from operator import mul
    from functools import reduce

    return reduce(mul,range(1,n+1))

fact(5)
# It is equivalent to fact = timed(logged(fact))

2025-06-17 11:56:37.001234-04:00 called fact
fact(5) took 0.000547s to run


120

## Application 3 : Memoization

In [69]:
# As we have seen fibonacci series using recursion takes lot of time.This is becuase we are calculating same fibonacci number many times.
# So to overcome this we can use a concept called memoization. Instead of calculate it so many times we can calculate it first and store it in some data structure and use it for later.

def fib(n):

    print("Claculating fib({0})".format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [ ]:
fib(5)

# In this output we can see fib(2), fib(1) is calculated so many times. So to overcome it we can use concept called memoization

Claculating fib(5)
Claculating fib(4)
Claculating fib(3)
Claculating fib(2)
Claculating fib(1)
Claculating fib(2)
Claculating fib(3)
Claculating fib(2)
Claculating fib(1)


5

In [73]:
# Here iam implementing it by using a simple class

class Memoization:

    def __init__(self):

        self.cache = {1:1,2:1}

    def fib(self,n):

        if n not in self.cache:
            print("calculating fib({0})".format(n))
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]

In [ ]:
# Now lets create object for this class and call fib function

f = Memoization()
f.fib(5)

# In the output we can see that we just calculated only 3 things

calculating fib(5)
calculating fib(4)
calculating fib(3)


5

In [ ]:
f.fib(10)

# Here we have calculated only 10,9,8,7,6 becasue until 5 we have already calculated and stored it in cache.

calculating fib(10)
calculating fib(9)
calculating fib(8)
calculating fib(7)
calculating fib(6)


55

In [79]:
# As we know all the simple classes can be written as closures. Now lets write the above class as a closure.

def fib():

    cache = {1:1,2:1}

    def calculate_fib(n):

        if n not in cache:
            print("Calaculating fib({0})".format(n))
            cache[n] = calculate_fib(n-1) + calculate_fib(n-2)

        return cache[n]
    return calculate_fib

f = fib()
f(5)

Calaculating fib(5)
Calaculating fib(4)
Calaculating fib(3)


5

In [80]:
f(10)

Calaculating fib(10)
Calaculating fib(9)
Calaculating fib(8)
Calaculating fib(7)
Calaculating fib(6)


55

In [81]:
# So now lets convert this closure into a decorator. As we know decorator takes a function and enhance the behaviour of the function

def memoize(fn):

    from functools import wraps
    cache = dict()

    @wraps(fn)
    def inner(n):

        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    return inner

# This function is general memoize function which actually stores all the calculated values.

In [82]:
# Now lets define the fib function and decorate it

@memoize

def fib(n):

    print("Calculating fib({0})".format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [83]:
fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


5

In [ ]:
# Now lets see the fib(10)
fib(10)
# We can see it as used the pre-calulated values of fibonacci series stored in cache.

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)


55

In [86]:
# This decorator can be used for any other function which need s memoization technique

# We know factorial also requires memoization.So lets do memoization for factorial

@memoize
def fact(n):
    print("Calculating fact({0})".format(n))
    return 1 if n <=1 else n*fact(n-1)

fact(5)

Calculating fact(5)
Calculating fact(4)
Calculating fact(3)
Calculating fact(2)
Calculating fact(1)


120

In [87]:
fact(10)

Calculating fact(10)
Calculating fact(9)
Calculating fact(8)
Calculating fact(7)
Calculating fact(6)


3628800

In [96]:
# So we have seen how decorators can be used for memoization technique. But we know this cache is unlimited which means we can store numerous numbers.

# If you have unlimited cache, we might end up the memory overflow errors. To overcome this error we need to use limited cache. But it is complicated to code this.

# But python has builtin decorator called lru_cache which performs memoization and limited caching. It is in functools. It is also a paramaterized decorator

from functools import lru_cache

@lru_cache(maxsize = 8) # Here iam limiting cache size is 8
def fib(n):
    print("calculating fib({0})".format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [97]:
fib(8)

calculating fib(8)
calculating fib(7)
calculating fib(6)
calculating fib(5)
calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)


21

In [ ]:
# Now lets calculate fib(16)

fib(16)

# Here it is calcualted only from 9 to 16 becuase upto 8 are stored in cache

calculating fib(16)
calculating fib(15)
calculating fib(14)
calculating fib(13)
calculating fib(12)
calculating fib(11)
calculating fib(10)
calculating fib(9)


987

In [ ]:
# So now again calcualte fib(8)

fib(8)

# If you see it is again calculated from 1 to 8. Since we have limited cache size to 8 , at first it is stored the values of 8 and when run fib(16),
# its starts deleting the oldest one first and stores the newest one. So like this its start deleting fib(1) when fib(9) got calculated. Like that,
# its deleted 1 to 8 to store fib(9) to fib(16) 

calculating fib(8)
calculating fib(7)
calculating fib(6)
calculating fib(5)
calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)


21

In [100]:
# If you calculate fib(16) again it starts calculating them from 9 to 16.

fib(16)

calculating fib(16)
calculating fib(15)
calculating fib(14)
calculating fib(13)
calculating fib(12)
calculating fib(11)
calculating fib(10)
calculating fib(9)


987